In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import datetime
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

GK_GREEN = '#21B534'
GK_GREEN_LIGHT = '#07f596'
GK_PURPLE = '#7494EA'
GK_BLUE = '#19535F'
RED = "#f55b5b"
DARK_RED = "#db0000"
GREY = "#a6a6a6"
BLUE = "#4287f5"
LIGHT_GREEN = '#88D39B'

MAJOR = 'MAJOR'
MINOR = 'MINOR'
PATCH = 'PATCH'
PINNED = 'PINNED'
NA_RELEASE_TYPE = pd.NA
DEP = 'Dependency'
DEV_DEP = 'Dev Dependency'

COLOUR_PALETTE = sns.color_palette('Greens_r')
GK_GREEN = COLOUR_PALETTE[1]
TWO_COLOUR_PALETTE = [GK_GREEN, GK_PURPLE]
THREE_COLOUR_PALETTE = [GK_GREEN, GK_PURPLE, GK_BLUE]
FOUR_COLOUR_PALETTE = THREE_COLOUR_PALETTE + [LIGHT_GREEN]


###################
# Helper functions
###################
def get_issue(issues, id):
    return issues[issues['issue_id'] == id]

def get_issue_url(issues, id):
    return issues[issues['issue_id'] == id]['issue_url']

def get_comment(comments, id):
    return comments[comments['comment_id'] == id]

def get_comments_for_issue(comments, id):
    return comments[comments['comment_issue_id'] == id]

def get_event(events, id):
    return events[events['event_id'] == id]

def calculate_percent(numer, denom):
    return round((numer/denom)*100, 2)

def get_update_type(prev_ver, new_ver):
    if pd.isnull(prev_ver) or pd.isnull(new_ver):
        return NA_RELEASE_TYPE
    try:
        prev_split = prev_ver.split('.')
        new_split = new_ver.split('.')
        if int(new_split[0]) > int(prev_split[0]):
            return MAJOR
        elif int(new_split[1]) > int(prev_split[1]):
            return MINOR
        elif int(new_split[2]) > int(prev_split[2]):
            return PATCH
        else:
            return NA_RELEASE_TYPE
    except Exception as e:
        return NA_RELEASE_TYPE
    
def get_update_type_v2(prev_ver, new_ver):
    if pd.isnull(prev_ver) or pd.isnull(new_ver):
        return NA_RELEASE_TYPE
    try:
        prev_split = prev_ver.split('.')
        prev_major = prev_split[0]
        prev_minor = prev_split[1]
        prev_patch = prev_split[2]
        new_split = new_ver.split('.')
        new_major = new_split[0]
        new_minor = new_split[1]
        new_patch = new_split[2]
        if new_major != prev_major:
            return MAJOR
        elif new_minor != prev_minor:
            return MINOR
        elif new_patch != prev_patch:
            return PATCH
        else:
            return NA_RELEASE_TYPE
    except Exception as e:
        return NA_RELEASE_TYPE
    
def get_issue_id_for_commit(events, issues, commit):
    try:
        event_id = commit.commit_event_id
        event = get_event(events, event_id)
        if event.size == 0:
            return np.nan
        issue = get_issue(issues, event.event_issue_id.values[0].astype(np.int64))
        if issue.size == 0:
            return np.nan
        return issue.issue_id.values[0].astype(np.int64)
    except Exception:
        return np.nan

/Users/BenRombaut/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
